## Linear-Exponential Adaptation Model 2 Parameter

In [8]:
'''

Purpose of code is to loop through hazard model data and generate LEA-fitted model based on observed h(t)_filtered_0.9 with 2 parameters.

If you would like to change the filtering, in the source file, there is an option for zero filtering. Please utilize the method that yeilds the most
significant and meaningful results.

INPUT: Hazard_Model_Data_1.csv

OUTPUT: Charts, results dataframe.

'''


import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt

df = pd.read_csv('Datasets/Hazard_Model_Data_1.csv')

# 2 parameter hazard function


def hazard_model(t, a_m, p):
    return t * a_m * np.exp(-t / p)


# objective function
def objective(params, t, y_observed):
    a_m, p = params
    y_estimated = hazard_model(t, a_m, p)
    return np.sum((y_observed - y_estimated) ** 2)


states = []
mobility_types = []
a_m_values = []
p_values = []
r2_values = []


for (state, mobility_type), group in df.groupby(['state', 'mobility_type']):
    t = np.array(np.arange(len(group['t'])))
    y_observed = group['h(t)_filtered_0.9'].values

    init_params = [0.1, 10]

    res = minimize(objective, init_params, args=(
        t, y_observed), bounds=((0, None), (0.00000001, None)))
    a_m_estimated, p_estimated = res.x

    y_predicted = hazard_model(t, a_m_estimated, p_estimated)

    SSR = np.sum((y_observed - y_predicted) ** 2)
    SST = np.sum((y_observed - np.mean(y_observed)) ** 2)
    R2 = 1 - (SSR / SST)

    '''plt.figure(figsize=(10, 6))
    plt.plot(t, y_observed, 'o-', label='Observed')
    plt.plot(t, hazard_model(t, a_m_estimated, p_estimated),
             'x-', label='Fitted Hazard Model')
    plt.xlabel('t')
    plt.ylabel('h(t)')
    plt.title(f'Observed vs. Fitted Hazard Model for {state}, {mobility_type}')
    plt.legend()
    plt.grid(True)
    plt.show()'''

    states.append(state)
    mobility_types.append(mobility_type)
    a_m_values.append(a_m_estimated)
    p_values.append(p_estimated)
    r2_values.append(R2)

results_df = pd.DataFrame({
    'State': states,
    'Mobility Type': mobility_types,
    'a_m': a_m_values,
    'p': p_values,
    'R^2': r2_values
})

results_df.to_csv("Results/LEA_2.csv")


## Linear-Exponential Adaptation Model 3 Parameter

In [9]:
'''

Purpose of code is to loop through hazard model data and generate LEA-fitted model based on observed h(t)_filtered_0.9 with 3 parameters.

If you would like to change the filtering, in the source file, there is an option for zero filtering. Please utilize the method that yeilds the most
significant and meaningful results.

INPUT: Hazard_Model_Data_1.csv

OUTPUT: Charts, results dataframe.

'''

import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# Read the data
df = pd.read_csv('Datasets/Hazard_Model_Data_1.csv')

# Define the custom hazard function


def hazard_model(t, a, s, p):
    return a * t * np.exp(- (t / p)**s * (1/s))

# Objective function for fitting


def objective(params, t, y_observed):
    a, s, p = params
    y_estimated = hazard_model(t, a, s, p)
    return np.sum((y_observed - y_estimated) ** 2)


# Initialize empty lists to store results
states = []
mobility_types = []
a_values = []
p_values = []
s_values = []
r2_values = []

# Loop over unique combinations of state and mobility_type
for (state, mobility_type), group in df.groupby(['state', 'mobility_type']):
    t = np.array(np.arange(len(group['t'])))
    y_observed = group['h(t)_filtered_0.9'].values

    # Initial guesses for a, s, and p
    init_params = [y_observed[0], 1, t[np.argmax(y_observed)]]

    # Perform optimization to fit the custom hazard model
    res = minimize(objective, init_params, args=(t, y_observed),
                   bounds=((0, None), (0.1, 26.98240641), (0.00000001, None)))  # Note: upper bound for s is determined by top 5% of s
    a_estimated, s_estimated, p_estimated = res.x

    y_predicted = hazard_model(t, a_estimated, s_estimated, p_estimated)

    # Calculate R^2
    SSR = np.sum((y_observed - y_predicted) ** 2)
    SST = np.sum((y_observed - np.mean(y_observed)) ** 2)
    R2 = 1 - (SSR / SST)

    # Plot the observed data and the fitted hazard model curve
    '''plt.figure(figsize=(10, 6))
    plt.plot(t, y_observed, 'o-', label='Observed')
    plt.plot(t, hazard_model(t, a_estimated, s_estimated,
             p_estimated), 'x-', label='Fitted Hazard Model')
    plt.axvline(p_estimated, color='r', linestyle='--',
                label=f'Peak at t={p_estimated:.5f}')
    plt.xlabel('t')
    plt.ylabel('h(t)')
    plt.title(f'Observed vs. Fitted Hazard Model for {state}, {mobility_type}')
    plt.legend()
    plt.grid(True)

    # Display values of a, s, p, and R^2
    plt.text(0.7 * t[-1], 0.7 * max(y_observed), f'a = {a_estimated:.5f}\ns = {s_estimated:.5f}\np = {p_estimated:.5f}\nR^2 = {R2:.5f}',
             fontsize=10, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))

    plt.show()'''

    states.append(state)
    mobility_types.append(mobility_type)
    a_values.append(a_estimated)
    p_values.append(p_estimated)
    s_values.append(s_estimated)
    r2_values.append(R2)

# Compile results into a dataframe
results_df = pd.DataFrame({
    'State': states,
    'Mobility Type': mobility_types,
    'a': a_values,
    'p': p_values,
    's': s_values,
    'R^2': r2_values
})

results_df.to_csv("Results/LEA_3.csv")


## LEA 3 Detailed Results Output

In [10]:
'''

Purpose of code is to export a detailed profile of h(t) fit by the LEA model. 

'''


import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# Read the data
df = pd.read_csv('Datasets/Hazard_Model_Data_1.csv')

# Define the custom hazard function


def hazard_model(t, a, s, p):
    return a * t * np.exp(- (t / p)**s * (1/s))

# Objective function for fitting


def objective(params, t, y_observed):
    a, s, p = params
    y_estimated = hazard_model(t, a, s, p)
    return np.sum((y_observed - y_estimated) ** 2)


# Lists to store results for the detailed dataframe
detailed_states = []
detailed_mobility_types = []
detailed_t = []
detailed_observed = []
detailed_fitted = []

# Loop over unique combinations of state and mobility_type
for (state, mobility_type), group in df.groupby(['state', 'mobility_type']):
    t = np.array(np.arange(len(group['t'])))
    y_observed = group['h(t)_filtered_0.9'].values

    # Initial guesses for a, s, and p
    init_params = [y_observed[0], 1, t[np.argmax(y_observed)]]

    # Perform optimization to fit the custom hazard model
    res = minimize(objective, init_params, args=(t, y_observed),
                   bounds=((0, None), (0.1, 26.98240641), (0.00000001, None)))  # Note: upper bund for s is determined by top 5% of s
    a_estimated, s_estimated, p_estimated = res.x

    y_predicted = hazard_model(t, a_estimated, s_estimated, p_estimated)

    # Append results to detailed lists
    detailed_states.extend([state] * len(t))
    detailed_mobility_types.extend([mobility_type] * len(t))
    detailed_t.extend(t)
    detailed_observed.extend(y_observed)
    detailed_fitted.extend(y_predicted)

# Compile the detailed results into a dataframe
detailed_df = pd.DataFrame({
    'State': detailed_states,
    'Mobility Type': detailed_mobility_types,
    't': detailed_t,
    'Observed h(t)': detailed_observed,
    'Fitted h(t)': detailed_fitted
})

detailed_df.to_csv('Results/fitted_hazard_detailed.csv')


## Weibull

In [7]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# Read the data
df = pd.read_csv('Datasets/Hazard_Model_Data_1.csv')

# Define the Weibull function


def weibull(t, A, B):
    return B / A * (t / A) ** (B - 1)

# Define the objective function for fitting


def objective(params, t, y_observed):
    A, B = params
    y_estimated = weibull(t, A, B)
    return np.sum((y_observed - y_estimated) ** 2)


# Create an empty dataframe to store results
results_df = pd.DataFrame(columns=['state', 'mobility_type', 'A', 'B', 'R2'])

# Loop over unique combinations of state and mobility_type
for (state, mobility_type), group in df.groupby(['state', 'mobility_type']):

    t = np.array(np.arange(len(group['t'])))
    y_observed = group['h(t)_filtered_0.9'].values

    # Initial guesses for A and B
    init_params = [1, 1]

    # Perform optimization to fit the Weibull distribution
    res = minimize(objective, init_params, args=(t, y_observed))
    A_estimated, B_estimated = res.x

    y_predicted = weibull(t, A_estimated, B_estimated)

    # Calculate R^2
    SSR = np.sum((y_observed - y_predicted) ** 2)
    SST = np.sum((y_observed - np.mean(y_observed)) ** 2)
    R2 = 1 - (SSR / SST)

    # Add results to dataframe
    results_df = results_df.append({'state': state, 'mobility_type': mobility_type,
                                   'A': A_estimated, 'B': B_estimated, 'R2': R2}, ignore_index=True)

    # Plot the observed data and the fitted Weibull curve
    '''plt.figure(figsize=(10, 6))
    plt.plot(t, y_observed, 'o-', label='Observed')
    plt.plot(t, weibull(t, A_estimated, B_estimated), 'x-', label='Fitted Weibull')
    plt.xlabel('t')
    plt.ylabel('h(t)')
    plt.title(f'Observed vs. Fitted Weibull Distribution for {state}, {mobility_type}')
    plt.legend()
    plt.grid(True)
    plt.show()'''

# Display the results
print(results_df)
results_df.to_csv('Results/fitted_weibull.csv')


       state                                      mobility_type           A  \
0    Alabama  grocery_and_pharmacy_percent_change_from_baseline   28.841722   
1    Alabama                 parks_percent_change_from_baseline   67.140957   
2    Alabama           residential_percent_change_from_baseline   28.618653   
3    Alabama  retail_and_recreation_percent_change_from_base...   52.439109   
4    Alabama      transit_stations_percent_change_from_baseline   54.717666   
..       ...                                                ...         ...   
307  Wyoming                 parks_percent_change_from_baseline   54.560914   
308  Wyoming           residential_percent_change_from_baseline   28.164201   
309  Wyoming  retail_and_recreation_percent_change_from_base...   33.017786   
310  Wyoming      transit_stations_percent_change_from_baseline   42.401589   
311  Wyoming            workplaces_percent_change_from_baseline  109.767107   

            B        R2  
0    1.249202  0.318761  